# 1. generate training samples (scenes)

In [1]:
import sys 
from ruixuan.turning_scene import *
from l5kit.rasterization.rasterizer_builder import _load_metadata
import time
import pickle
import math
from collections import deque
import numpy as np
from sklearn.mixture import BayesianGaussianMixture
from tabulate import tabulate
from utils import Gibbs_sampling
from config import Config
from visualizer import plot_scene_on_grid

In [2]:
pxy= pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_Pxy.pickle",'rb'))
pxz= pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_Pxz.pickle",'rb'))
pyzx= pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_Pyzx.pickle",'rb'))

veh_model = pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_veh_model.pickle",'rb'))
ped_model = pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_ped_model.pickle",'rb'))
cyc_model = pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_cyc_model.pickle",'rb'))

poolv = pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_veh_pool.pickle",'rb'))
poolp = pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_ped_pool.pickle",'rb'))
poolc = pickle.load(open("/home/lab1/repo/planning/saved_gibbs/8kfb_cyc_pool.pickle",'rb'))

/home/lab1/miniconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator BayesianGaussianMixture from version 0.24.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
sampling_res = Gibbs_sampling(max_scene=20, Pxy=pxy, Pxz=pxz, Pyzx=pyzx, poolv=poolv, poolp=poolp, poolc=poolc,\
                             veh_model = veh_model, ped_model=ped_model, cyc_model=cyc_model,\
                             max_n_veh=10, max_n_ped=5,max_n_cyc=5)
cfg = Config()

In [4]:
scene_data  = sampling_res[18]
veh_s = scene_data['veh']
np.random.shuffle(veh_s)
ped_s = scene_data['ped']
cyc_s = scene_data['cyc']
print(veh_s.shape, ped_s.shape, cyc_s.shape)

(8, 20, 2) (5, 20, 2) (1, 20, 2)


In [5]:

# plot_scene_on_grid('8KfB', lane_list=cfg.lane_list, \
#                            grid_boundary=cfg.grid_boundary, Test=cfg.Test, veh=veh_s[:], ped=ped_s, cyc=cyc_s)

In [6]:
import itertools 
# x=list(itertools.product([1,2], [2,3]))
# x

# 2. construct graph

In [7]:

def pred_veh(pred_model_v, traj):
    pass 
def pred_risk(risk_model, traj):
    pass
def pred_ped(pred_model_p, traj):
    pass 
def pred_cyc(pred_model_c, traj):
    pass 

def compute_vweight(target, veh):
    """
    computes the edge weights between target and veh
    """
    pos = target.pos
    pass
    
def compute_pcweight(target, pc):
    """
    computes the edge weights between target and ped/cyc
    """
    pos = target.pos
    pass



In [22]:
# planned subject: 
# pick a left/right/straight trajectory to plan

class TargetNode():
    """
    build target node
    """
    def __init__(self, traj):
        self.start=traj[10] # position at 5th second
        self.goal=traj[-1] # last point
        self.pos=self.start
        self.task=np.array(self.start,self.goal)
  
        self.a,self.v = self._get_av(traj[8], traj[9], traj[10])
        self.theta = self._get_theta(traj[9], traj[10])
        self.t=0
        
        self.history=[]
        self.history.append([self.t, self.pos, self.a, self.theta])
        
        
    def _get_av(self, prev2, prev, curr):
        v10 = np.linalg.norm(curr - prev)/0.5
        v9 = np.linalg.norm(prev - prev2)/0.5
        a = (v10-v9)/0.5
        return a, v10
    
    def _get_theta(self, prev,curr):
        try:
            tan = (curr[1]-prev[1])/(curr[0]-prev[0])
            return math.atan(tan)
        except:
            return 90
        
class Node():
    """
    build other nodes
    """
    def __init__(self,traj, pred, agent='veh',risk=0.5):
        self.traj=traj
        self.pred=pred
        self.agent=agent
        if agent=='veh':
            self.risk=risk
        else:
            # if agent==veh, there is risk, otherwise the risk is not meaningful
            self.risk=-1


In [9]:




class Graph():
    def __init__(self, sample, models, lane_boundary):
        self.sample=sample
        self.graph={}
        self.vmodel=models[0]
        self.pmodel=models[1]
        self.cmodel=models[2]
        self.riskmodel=models[3]
        self.lane_boundary=lane_boundary
        
        self.action_dict = list(itertools.product(range(-3,3,0.1), range(0,180,1)))
        
        return self._build_graph_
        
    def _build_graph_(self):
        subject = sample['veh'][0]
        self.target= TargetNode(subject)
        self.env_veh=[]
        self.env_ped=[]
        self.env_cyc=[]
        self.edges=[]
        
        # build nodes/vertices
        for i in range(1, len(self.sample['veh'])):
            traj = self.sample['veh'][i]
            pred_traj = pred_veh(self.vmodel, traj)
            risk = pred_risk(self.riskmodel, traj)
            node = Node(traj,pred_traj,'veh',risk)
            self.env_veh.append(node)
            
        
        for traj in samples['cyc']:
            pred_traj = pred_cyc(self.cmodel,traj)
            node=Node(traj, pred_traj,'cyc')
            self.env_cyc.append(node)
            
    
        for traj in samples['ped']:
            pred_traj = pred_cyc(self.pmodel,traj)
            node=Node(traj, pred_traj,'ped')
            self.env_ped.append(node)
            
        self._update_edges()
        # build edges
        
        return self
        
    def _update_edges(self):
        
        for node in self.env_veh:
            weight = compute_vweight(self.target, node)
            self.edges.append(weight)
            
        for node in self.env_cyc:
            weight = compute_pcweight(self.target, node)
            self.edges.append(weight)
            
        for node in self.env_ped:
            weight = compute_pcweight(self.target, node)
            self.edges.append(weight)
    
    def _take_action(self, policy):
        
        # action = self.take_action(policy)
        # return action
        pass
    
    def is_crash(self):
        """
        check if crash happens when graph is updated, or hit lane_boundary inside intersection
        """
        pass

    def step(self, action, policy):
        """
        update graph, move to next state
        action space [-3, 3] * [0,180] => 60*180 action space
        """
        acc, theta = self.action_dict[action]
        distance = self.target.v*0.5+0.5*acc*0.25
        radian = theta*math.pi/180
        dx, dy = distance*math.cos(radian), distance*math.sin(radian)
        
        #update state
        self.target.pos =[self.target.pos[0]+dx, self.target.pos[1]+dy]
        self.target.v = self.target.v+acc*0.5
        self.target.a = acc
        self.target.theta=theta
        self.target.t +=1

        self.target.history.append([self.target.t, self.target.pos, self.target.a, self.target.theta])
        # update the state
        self._update_edges()  #maybe put to later
        distance_to_goal = np.linalg.norm(self.target.pos - self.target.goal, axis=1)
        
        # compute reward
        # stop crateria: #step longer than 5 seconds, reached goal, crash with other agents
        if  distance_to_goal<1:
            # update reward
            pass
        
        elif self.t>10:
            # update reward
             #return 'done'
            pass
        elif is_crash():  
            # this can be moved to update edge part by giving a huge loss, is the loss pass the threshold then done
            # update reward
            pass
        else:
            # update reward
            pass
        
    
    def _get_reward(self):
        """
        compute cost of the current state, A*sum_t sum_edge c_pi +B*c_goal +C* c_smoothness
        """
        pass
        


# map graph to states with clustering to fit into reinforcement learning algorithm

In [25]:
from tslearn.metrics import dtw,cdist_dtw
# 
a=[[2,3], 1,2]
b=[[2,4],2,2]
s= cdist_dtw(a,b, n_jobs=-1)
amin = np.amin(s, axis=1)
print(s,amin, sum(amin))


[[1.         1.         1.        ]
 [3.16227766 1.         1.        ]
 [2.         0.         0.        ]] [1. 1. 0.] 2.0


In [12]:
def get_sim(a,b):
    s= cdist_dtw(a,b, n_jobs=-1)
    amin = np.amin(s, axis=1)
#     print(s,amin, sum(amin))
    # sum the diagno of the cdist_dtw matrix
    return sum(amin)

def compute_similarity(g1, g2):
    """
    compute the similarity between two graphs
    final similarity between two graphs:
    s = w1* similarity_between_target_nodes + 
    w2*[similarity_between_veh_nodes +
    similarity_between_ped_nodes +
    similarity_between_cyc_nodes] +
    w3* similarity_between_edges
    
    helper for formulating states, give a good reward signal, the states should transit closer to goal
    """
    # focusing on subject, but the similarity is regularized by envronment
    # target_sim
    target_task_s = get_sim(g1.target.goal,g2.target.goal)
    g1t_attr = [g1.target.pos, g1.target.a, g1.target.theta/180]
    g2t_attr = [g2.target.pos, g2.target.a, g2.target.theta/180]
    target_attr_s =get_sim(g1t_attr, g2t_attr)
    subject_sim = target_task_s + target_attr_s
    # veh_sim
    veh_traj =[]
    for node in g1.env_veh:
        
    
    
    